## 13. RDD 고급 개념

- 집계와 키-값 형태의 RDD
- 사용자 정의 파티셔닝
- RDD 조인


In [1]:
# 12장에서 사용된 데이터 불러오기 
myCollection = 'Spark The Definitive Guide: Big Data Processing Made Simple'.split(' ')
words = spark.sparkContext.parallelize(myCollection, 2)
words.collect()

NameError: name 'spark' is not defined

### 13.1 키-값 형태의 기초(RDD)
- ByKey가 있다면 pariRDD 타입만 사용할 수 있음
- RDD에 맵 연산을 수행해 키-값 구조로 만듦

In [2]:
words.map(lambda word: (word.lower(), 1)).collect()

NameError: name 'words' is not defined

#### 13.1.1 keyBy
- 현재 값으로부터 키를 생성하는 KeyBy 함수를 사용하면 동일한 결과 반환
- 스파크는 원본 단어를 생성된 RDD의 값으로 유지

In [3]:
keyword = words.keyBy(lambda word: word.lower()[0])
keyword.collect()

NameError: name 'words' is not defined

#### 13.1.2 값 매핑하기 
- 튜플 형태의 데이터를 사용하면 스파크는 튜플의 첫 번째 요소를 키로, 두 번째 요소를 값으로 추정함
- mapValues를 쓰면 값 수정 시 발생할 수 있는 오류를 미리 방지할 수 있음

In [6]:
keyword.mapValues(lambda word: word.upper()).collect()

[('s', 'SPARK'),
 ('t', 'THE'),
 ('d', 'DEFINITIVE'),
 ('g', 'GUIDE:'),
 ('b', 'BIG'),
 ('d', 'DATA'),
 ('p', 'PROCESSING'),
 ('m', 'MADE'),
 ('s', 'SIMPLE')]

- flatMap 함수를 사용해 반환되는 결과의 각 로우가 문자를 나타내도록 확장할 수 있음 

In [8]:
keyword.flatMapValues(lambda word: word.upper()).collect()[0:10]

[('s', 'S'),
 ('s', 'P'),
 ('s', 'A'),
 ('s', 'R'),
 ('s', 'K'),
 ('t', 'T'),
 ('t', 'H'),
 ('t', 'E'),
 ('d', 'D'),
 ('d', 'E')]

#### 13.1.3 키와 값 추출하기

In [9]:
print('키')
print(keyword.keys().collect())
print('값')
print(keyword.values().collect())

키
['s', 't', 'd', 'g', 'b', 'd', 'p', 'm', 's']
값
['Spark', 'The', 'Definitive', 'Guide:', 'Big', 'Data', 'Processing', 'Made', 'Simple']


#### 13.1.4 lookup
- 특정 키에 대한 결과를 찾을 수 있음

In [10]:
keyword.lookup('s')

['Spark', 'Simple']

#### 13.1.5 sampleByKey
- 근사치나 정확도를 이용해 키를 기반으로 RDD 샘플 생성
- 부분 샘플링 할 수 있고, 비복원 추출도 가능


In [17]:
# 매번 추출 결과가 다르게 나온다.
import random
distinctChars = words.flatMap(lambda word: list(word.lower())).distinct().collect()
sampleMap = dict(map(lambda c: (c,random.random()), distinctChars))
words.map(lambda word: (word.lower()[0], word)).sampleByKey(True, sampleMap, 6).collect()

[('t', 'The'), ('t', 'The'), ('g', 'Guide:')]

- sampleByKeyExact 매서드는 99.99% 신뢰도를 가진 모든 키값에 대해 RDD를 추가로 처리함
- 비복원 추출을 사용한다면 샘플 크기를 보장하기 위해 RDD를 한번 더 통과해야 하고, 복원 추출을 사용한다면 RDD를 두번 더 통과해야함.
- pyspark에는 구현 안되어 있는듯...

### 13.2 집계


In [22]:
chars = words.flatMap(lambda word: word.lower())
KVcharacters = chars.map(lambda letter: (letter, 1))

def maxFunc(left, right):
    return max(left, right)

def addFunc(left, right):
    return left + right

nums = sc.parallelize(range(1,31), 5)
nums.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30]

#### 13.2.1 countByKey
- 각 키의 아이템 수를 구하고 로컬 맵으로 결과를 수집함


In [23]:
KVcharacters.countByKey()

defaultdict(int,
            {'s': 4,
             'p': 3,
             'a': 4,
             'r': 2,
             'k': 1,
             't': 3,
             'h': 1,
             'e': 7,
             'd': 4,
             'f': 1,
             'i': 7,
             'n': 2,
             'v': 1,
             'g': 3,
             'u': 1,
             ':': 1,
             'b': 1,
             'o': 1,
             'c': 1,
             'm': 2,
             'l': 1})

#### 13.2.2 집계 연산 구현 방식 이해하기

- groupByKey
 - 각 키의 총 레코드를 수를 구하는 경우 groupByKey의 결과로 만들어진 그룹에 map 연산을 수행하는 방식을 추천


In [25]:
from functools import reduce
KVcharacters.groupByKey().map(lambda row: (row[0], reduce(addFunc, row[1]))).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

- 해당 방식에서 가장 큰 문제는 모든 익스큐터에서 함수를 적용하기 전에 해당 키와 관련된 모든 값을 메모리로 읽어 들여야하는것! 
- 심각하게 치우쳐진 키가 있다면 일부 파티션에만 엄청난 값을 가질 수 있기 때문에 OutOfMemoryError 발생
- 대규모 분산 환경에서는 심각한 에러 발생
- 그러므로 이 방식은 각 키에 대한 값의 크기가 일정하고 익스큐터에 할당된 메모리에서 처리 가능할 정도일 경우에만 사용

- reduceByKey
 - 각 파티션에서 리듀스 작업을 수행하기 때문에 훨씬 안정적이며 모든 값을 메모리에 유지하지 않아도 됨
 - 리듀스 과정을 제외한 모든 작업은 개별 워커에서 처리하기 때문에 연산 중에 셔플이 발생하지 않음
 - 키별 그룹 RDD를 반환함. 개별 요소들은 따로 정렬되지 않음, 따라서 순서가 중요할 때는 사용하지 않는 것이 좋음

In [26]:
KVcharacters.reduceByKey(addFunc).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

### 13.2.3 기타 집계 메서드
- 고급 집계 함수를 사용해 클러스터 노드에서 수행하는 집계를 아주 구체적이고 매우 세밀하게 제어할 수 있음

- aggregate
 - null 값이나 집계의 시작값이 필요하고 두 가지 파라미터를 사용함
 - 드라이버에서 최종 집계를 수행하기 때문에 성능에 약간의 영향을 끼침
 - 익스큐터가 너무 크면 OutOfMemoryError 발생 
 - treeAggregate는 동일한 작업이지만 처리 과정이 다름
   - 드라이버에서 최종 집계를 수행하기 전에 익스큐터끼리 트리를 형성해 집계 처리의 일부 하위 과정을 '푸시 다운' 방식으로 먼저 수행함
   - 집계 처리를 여러 단계로 구성하면 드라이버의 메모리를 모두 소비하는 현상을 막아줌

In [27]:
# 첫 번째 함수는 파티션 내에서 수행되고 두 번째 함수는 모든 파티션에 걸쳐 수행됨 
nums.aggregate(0, maxFunc, addFunc)

90

In [28]:
depth = 3
nums.treeAggregate(0, maxFunc, addFunc, depth)

90

- aggregateByKey
 - 파티션 대신 키를 기준으로 연산을 수행함
 

In [30]:
KVcharacters.aggregateByKey(0,addFunc,maxFunc).collect()

[('s', 3),
 ('p', 2),
 ('r', 1),
 ('h', 1),
 ('d', 2),
 ('i', 4),
 ('g', 2),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 3),
 ('k', 1),
 ('t', 2),
 ('e', 4),
 ('f', 1),
 ('n', 1),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

- combineByKEy
 - 집계함수 대신 컴바이너 사용
 - 키를 기준으로 연산을 수행하며 파라미터로 사용된 함수에 따라 값을 병행함
 - 여러 컴바이너의 결괏값을 병합해 결과를 반환함
 - 사용자 정의 파티셔너를 사용해 출력 파티션 수를 지정할 수 있음
 

In [33]:
def valToCombiner(value):
    return [value]

def mergeValuesFunc(vals, valToAppend):
    vals.append(valToAppend)
    return vals

def mergeCombinerFunc(vals1, vals2):
    return vals1 + vals2
outputPartitions = 6

KVcharacters.combineByKey(valToCombiner, mergeValuesFunc, mergeCombinerFunc, outputPartitions).collect()

[('s', [1, 1, 1, 1]),
 ('d', [1, 1, 1, 1]),
 ('l', [1]),
 ('v', [1]),
 (':', [1]),
 ('p', [1, 1, 1]),
 ('r', [1, 1]),
 ('c', [1]),
 ('k', [1]),
 ('t', [1, 1, 1]),
 ('n', [1, 1]),
 ('u', [1]),
 ('o', [1]),
 ('h', [1]),
 ('i', [1, 1, 1, 1, 1, 1, 1]),
 ('g', [1, 1, 1]),
 ('b', [1]),
 ('a', [1, 1, 1, 1]),
 ('e', [1, 1, 1, 1, 1, 1, 1]),
 ('f', [1]),
 ('m', [1, 1])]

- foldByKey
 - 결합 함수와 항등원인 '제로값'을 이용해 각 키의 값을 병합함
 - 제로값은 결과에 여러번 사용할 수 있으나 결과를 변경할 수는 없음(덧셈에서는 0, 뺄셈에서는 1)


In [37]:
KVcharacters.foldByKey(0, addFunc).collect()

[('s', 4),
 ('p', 3),
 ('r', 2),
 ('h', 1),
 ('d', 4),
 ('i', 7),
 ('g', 3),
 ('b', 1),
 ('c', 1),
 ('l', 1),
 ('a', 4),
 ('k', 1),
 ('t', 3),
 ('e', 7),
 ('f', 1),
 ('n', 2),
 ('v', 1),
 ('u', 1),
 (':', 1),
 ('o', 1),
 ('m', 2)]

### 13.3 cogroup
- 스칼라는 최대 3개, 파이썬은 최대 2개의 키-값 형태의 RDD를 그룹화할 수 있으며 각 키를 기준으로 결합함. 
- RDD에 대한 그룹 기반의 조인을 수행함
- 출력 파티션 수나 클러스터에 데이터 분산 방식을 정확하게 제어하기 위해 사용자 정의 파티션 함수를 파라미터로 사용할 수 있음

In [38]:
import random
distinctChars = words.flatMap(lambda word: word.lower()).distinct()
charRDD = distinctChars.map(lambda c: (c, random.random()))
charRDD2 = distinctChars.map(lambda c: (c, random.random()))

charRDD.cogroup(charRDD2).take(5)

[('s',
  (<pyspark.resultiterable.ResultIterable at 0x7f5c5da0f748>,
   <pyspark.resultiterable.ResultIterable at 0x7f5c5da3e0b8>)),
 ('p',
  (<pyspark.resultiterable.ResultIterable at 0x7f5c5da090f0>,
   <pyspark.resultiterable.ResultIterable at 0x7f5c5da3e128>)),
 ('r',
  (<pyspark.resultiterable.ResultIterable at 0x7f5c5da3e160>,
   <pyspark.resultiterable.ResultIterable at 0x7f5c5da3e198>)),
 ('i',
  (<pyspark.resultiterable.ResultIterable at 0x7f5c5da3e0f0>,
   <pyspark.resultiterable.ResultIterable at 0x7f5c5da3e208>)),
 ('g',
  (<pyspark.resultiterable.ResultIterable at 0x7f5c5da3e240>,
   <pyspark.resultiterable.ResultIterable at 0x7f5c5da3e278>))]

### 13.4 조인
- 구조적 API와 비슷한 조인 방식을 가지고 있지만, RDD는 사용자가 많은 부분에 관여해야함
- 출력 파티션 수나 사용자 정의 파티션 함수를 파라미터로 사용함 

#### 13.4.1 내부 조인
- 조인 방식
 1. fullOuterJoin
 2. leftOuterJoin
 3. rightOuterJoin
 4. cartesian(조인 키를 사용하지 않기 때문에 엄청 큰 출력 결과를 만들 수 있음)

In [39]:
keyedChars = distinctChars.map(lambda c: (c, random.random()))
outputPartitions = 10

print(KVcharacters.join(keyedChars).count())
print(KVcharacters.join(keyedChars, outputPartitions).count())

51
51


#### 13.4.2 zip
- 두개의 RDD를 결합하는 방식
- 동일한 길이의 두 개의 RDD를 지퍼를 잠그듯 연결할 수 있고, pairRDD 생성됨

In [41]:
numRange = sc.parallelize(range(9), 2)
words.zip(numRange).collect()

[('Spark', 0),
 ('The', 1),
 ('Definitive', 2),
 ('Guide:', 3),
 ('Big', 4),
 ('Data', 5),
 ('Processing', 6),
 ('Made', 7),
 ('Simple', 8)]

### 13.5 파티션 제어하기
- RDD를 사용하면 데이터가 클러스터 전체에 물리적으로 정확히 분산되는 방식을 정의할 수 있음
- 구조적 API와 가장 큰 차이점은 파티션 함수를 파라미터로 사용할 수 있다는 것

#### 13.5.1 coalesce
- 파티션을 재분배할 때 발생하는 데이터 셔플을 방지하기 위해 동일한 워커에 존재하는 파티션을 합치는 매서드

In [44]:
words.coalesce(1).getNumPartitions()

1

### 13.5.2 repartition
- 파티션 수를 늘리거나 줄일 수 있지만, 처리 시 노드 간의 셔플이 발생할 수 있음
- 파티션 수를 늘리면 맵 타입이나 필터 타입의 연산을 수행할 때 병렬 처리 수준을 높일 수 있음

In [46]:
words.repartition(10).collect()

['Spark',
 'The',
 'Definitive',
 'Guide:',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

#### 13.5.3 repartitionAndSortWithinPartitions
- 파티션을 재분배할 수 있고, 재분배된 결과 파티션의 정렬 방식을 지정할 수 있음
- 파티셔닝과 키 비교 모두 사용자가 지정할 수 있음

#### 13.5.4 사용자 정의 파티셔닝
- 저수준 API의 세부적인 구현 방식
- 페이지랭크는 사용자의 정의 파티셔닝을 이용해 클러스터의 데이터 배치 구조를 제어하고 셔플을 회피함.
- 데이터 치우침 같은 문제를 피하고자 클러스터 전체에 걸쳐 데이터를 균등하게 분배
- 사용자 정의 파티셔너를 사용하려면 구조적 API로 RDD를 얻고 사용자 정의 파티셔너를 적용한 다음 다시 DataFrame이나 Dataset으로 변환해야함.



In [49]:
df = spark.read.option('header','true').option('inferSchema', 'true').csv('file:///home/ubuntu/Spark-The-Definitive-Guide/data/retail-data/all')
rdd = df.coalesce(10).rdd
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



- HashPartitioner는 이산형 데이터
- RangePArtitioner는 연속형 데이터 
- 매우 큰 데이터나 심각하게 치우친 키를 다뤄야 하면 고급 파티셔닝 기능을 사용해야함. 
- 병렬성을 개선하고 실행 과정에서 OutOfMemoryError를 방지할 수 있도록 키를 최대한 분할해야함. 

In [52]:
def partitionFunc(key):
  import random
  if key == 17850 or key == 12583:
    return 0
  else:
    return random.randint(1,2)

keyedRDD = rdd.keyBy(lambda row: row[6])
keyedRDD\
  .partitionBy(3, partitionFunc)\
  .map(lambda x: x[0])\
  .glom()\
  .map(lambda x: len(set(x)))\
  .take(5)

[2, 4301, 4304]

### 13.6 사용자 정의 직렬화
- Kryo를 사용하면 더 빠르게 객체를 직렬화 할 수 있음
- 자바 직렬화보다 약 10배 이상 성능이 좋고 간결함
- spark.serralizer 설정으로 워커 노드 간 데이터 셔플링과 RDD를 직렬화해 디스크에 저장하는 용도로 사용할 시리얼라이저를 지정할 수 있음.
- 네트워크에 민감한 애플리케이션에서 사용할 것을 권장
- 스파크 2 버전부터는 단순 데이터 타입, 배열, 문자열 데이터 타입의 RDD를 셔플링하면 내부적으로 Kyro 시리얼라이저를 사용함